<a href="https://colab.research.google.com/github/Nilanshrajput/NER_SyferText/blob/master/lSTM_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/synalp/NER.git

Cloning into 'NER'...
remote: Enumerating objects: 3148, done.
remote: Total 3148 (delta 0), reused 0 (delta 0), pack-reused 3148
Receiving objects: 100% (3148/3148), 281.51 MiB | 38.66 MiB/s, done.
Resolving deltas: 100% (2066/2066), done.
Checking out files: 100% (189/189), done.


In [0]:

from __future__ import print_function
from collections import OrderedDict

import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
from torch import autograd

import time
import _pickle as cPickle

import urllib
import matplotlib.pyplot as plt


import os
import sys
import codecs
import re
import numpy as np
import spacy
from sklearn.metrics import f1_score

In [3]:
while 1:
  continue

KeyboardInterrupt: ignored

In [0]:

import torch
from torchtext import data
from torchtext.datasets import SequenceTaggingDataset
from torchtext.vocab import Vectors, GloVe

import numpy as np
import random
import logging
logger = logging.getLogger(__name__)


def conll2003_dataset(tag_type, batch_size, root='/content/NER/corpus/CoNLL-2003', 
                          train_file='eng.train', 
                          validation_file='eng.testa',
                          test_file='eng.testb',
                          convert_digits=True):
    """
    conll2003: Conll 2003 (Parser only. You must place the files)
    Extract Conll2003 dataset using torchtext. Applies GloVe 6B.200d and Char N-gram
    pretrained vectors. Also sets up per word character Field
    Parameters:
        tag_type: Type of tag to pick as task [pos, chunk, ner]
        batch_size: Batch size to return from iterator
        root: Dataset root directory
        train_file: Train filename
        validation_file: Validation filename
        test_file: Test filename
        convert_digits: If True will convert numbers to single 0's
    Returns:
        A dict containing:
            task: 'conll2003.' + tag_type
            iters: (train iter, validation iter, test iter)
            vocabs: (Inputs word vocabulary, Inputs character vocabulary, 
                    Tag vocabulary )
    """
    
    # Setup fields with batch dimension first
    inputs_word = data.Field(  lower=True)

    inputs_char_nesting = data.Field(tokenize=list)

    inputs_char = data.NestedField(inputs_char_nesting)
                        

    labels = data.Field(unk_token = None, is_target=True)

    fields = ([(('inputs_word', 'inputs_char'), (inputs_word, inputs_char))] + 
                [('labels', labels) if label == tag_type else (None, None) 
                for label in ['pos', 'chunk', 'ner']])

    # Load the data
    train, val, test = SequenceTaggingDataset.splits(
                                path=root, 
                                train=train_file, 
                                validation=validation_file, 
                                test=test_file,
                                separator=' ',
                                fields=tuple(fields))


    
    # Build vocab
    inputs_char.build_vocab(train.inputs_char, val.inputs_char, test.inputs_char)
    inputs_word.build_vocab(train.inputs_word, val.inputs_word, test.inputs_word, max_size=50000,
                        vectors= "glove.6B.300d")
    
    labels.build_vocab(train)
  

    # Get iterators
    train_iter, val_iter, test_iter = data.BucketIterator.splits(
                            (train, val, test), batch_size=batch_size, 
                            device=torch.device("cuda"))
    train_iter.repeat = False
    
    return {
        'task': 'conll2003.%s'%tag_type,
        'iters': (train_iter, val_iter, test_iter), 
        'vocabs': (inputs_word.vocab, inputs_char.vocab, labels.vocab), 
        'fields': (inputs_word,labels)
        }


In [0]:
dic = conll2003_dataset('ner', batch_size = 256)

In [74]:
a,_,_= dic['iters']

for i,t in enumerate(a):
  print(t)
  if i == 1:
    break

  


[torchtext.data.batch.Batch of size 64]
	[.labels]:[torch.LongTensor of size 44x64]
	[.inputs_word]:[torch.LongTensor of size 44x64]
	[.inputs_char]:[torch.LongTensor of size 64x44x15]
tensor([ 1646,   264,    20,  3672,  2015,   670, 26767,    17,    17,  1560,
        21912,   135,  2161,     1,     3,  2637,    17,     6,    14,     3,
         4314,     6,     1,   625,  1334,    65,    14,     1,     9,     1,
           98,    64,    31,   527, 12176,   107,   255,    15,  3865, 21049,
         4150,  2579,    25,  5961, 22291,  8596,    71,   177,    77,  1211,
         2513,     3,  6063,   248,   372, 19870,   397,     9,    44,  1027,
            3,   248,   547,     5])

[torchtext.data.batch.Batch of size 64]
	[.labels]:[torch.LongTensor of size 40x64]
	[.inputs_word]:[torch.LongTensor of size 40x64]
	[.inputs_char]:[torch.LongTensor of size 64x40x16]
tensor([ 2699,  4952,     3, 19028,   242,    24, 12126,    10,  8641,   939,
         1850,     5,     2,    24,  6911,  5

In [0]:
d_,_,label_=dic['vocabs']

In [76]:
len(label_)

9

In [77]:
label_.itos

['<pad>', 'O', 'I-PER', 'I-ORG', 'I-LOC', 'I-MISC', 'B-MISC', 'B-ORG', 'B-LOC']

In [78]:
label_.freqs.most_common()

[('O', 170524),
 ('I-PER', 11128),
 ('I-ORG', 10001),
 ('I-LOC', 8286),
 ('I-MISC', 4556),
 ('B-MISC', 37),
 ('B-ORG', 24),
 ('B-LOC', 11)]

In [0]:
train_iter,valid_iter,test_iter = dic['iters']
text_vocab,_,labels = dic['vocabs']
input_field,label_field = dic['fields']

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class LSTM_Tagger(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers = n_layers,
                            bidirectional = True,
                            dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        #pass text through embedding layer
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pass embeddings into LSTM
        outputs, (hidden, cell) = self.lstm(embedded)
        
        #outputs holds the backward and forward hidden states in the final layer
        #hidden and cell are the backward and forward hidden and cell states at the final time-step
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]
        
        #we use our outputs to make a prediction of what the tag should be
        predictions = self.fc(self.dropout(outputs))
        
        #predictions = [sent len, batch size, output dim]
        
        return predictions


In [0]:

INPUT_DIM = len(text_vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = len(labels)
N_LAYERS = 2

DROPOUT = 0.25
PAD_IDX = text_vocab.stoi[input_field.pad_token]
model = LSTM_Tagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS,
                        DROPOUT, 
                        PAD_IDX)

In [208]:
pretrained_embeddings = text_vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([26872, 300])


In [209]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0702, -0.0498,  0.2675,  ...,  0.2944, -0.4164,  0.3202],
        [-0.1881, -0.4332, -0.4754,  ...,  0.6583,  0.2311,  0.1486],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [210]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0702, -0.0498,  0.2675,  ...,  0.2944, -0.4164,  0.3202],
        [-0.1881, -0.4332, -0.4754,  ...,  0.6583,  0.2311,  0.1486],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [0]:
device = torch.device("cuda")

In [0]:
optimizer = torch.optim.Adam(model.parameters(),lr = 2e-5)

In [0]:
TAG_PAD_IDX = labels.stoi[label_field.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

In [0]:
def metric_f1(preds, y, tag_pad_idx):
  preds=preds.to('cpu')
  y = y.to('cpu')
  max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
  non_pad_elements = (y != tag_pad_idx).nonzero()
  shape = max_preds[non_pad_elements].shape[0]
  #print(max_preds[non_pad_elements])
  #print(max_preds[non_pad_elements].view(shape).shape)
  #print(y[non_pad_elements])
  #print(y[non_pad_elements].view(shape).shape)
  return f1_score(y[non_pad_elements].detach().view(shape).numpy(),max_preds[non_pad_elements].detach().view(shape).numpy(),average='macro')

In [0]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)

def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch.inputs_word
        tags = batch.labels
        
        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        predictions = model(text)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        f1 = metric_f1(predictions, tags, tag_pad_idx)
        
        loss.backward()
        clip_gradient(model, 1e-1)
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_f1 += f1
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator)

In [0]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.inputs_word
            tags = batch.labels
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)
            f1 = metric_f1(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_f1 += f1
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [220]:
N_EPOCHS = 100

best_valid_f1 = float(0)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_iter, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc, valid_f1 = evaluate(model, valid_iter, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_f1 > best_valid_f1:
        best_valid_f1 = valid_f1
        torch.save(model.state_dict(), 'best_f1_bilstmmodel.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train_F1: {train_f1}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Valid_F1: {valid_f1}')

Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 2.037 | Train Acc: 77.59% | Train_F1: 0.14019553411928612
	 Val. Loss: 1.926 |  Val. Acc: 78.30% | Valid_F1: 0.16747860084634342
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 1.391 | Train Acc: 83.34% | Train_F1: 0.15772370645433126
	 Val. Loss: 1.181 |  Val. Acc: 78.31% | Valid_F1: 0.16957552358501618
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.785 | Train Acc: 83.34% | Train_F1: 0.16101099029517615
	 Val. Loss: 1.083 |  Val. Acc: 78.31% | Valid_F1: 0.16957552358501618
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.717 | Train Acc: 83.34% | Train_F1: 0.16239746966517668
	 Val. Loss: 1.027 |  Val. Acc: 78.32% | Valid_F1: 0.16796096340478545
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.667 | Train Acc: 83.34% | Train_F1: 0.1609999266921179
	 Val. Loss: 0.983 |  Val. Acc: 78.33% | Valid_F1: 0.16796526250987526
Epoch: 06 | Epoch Time: 0m 8s
	Train Loss: 0.630 | Train Acc: 83.34% | Train_F1: 0.16052968701456163
	 Val. Loss: 0.957 |  Val. Acc: 78.36% |

In [222]:
model.load_state_dict(torch.load('best_f1_bilstmmodel.pt'))

test_loss, test_acc,test_f1 = evaluate(model, test_iter, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}% | Test_F1: {test_f1}')

Test Loss: 0.318 |  Test Acc: 94.57% | Test_F1: 0.7343369025854302


In [0]:
def tag_sentence(model, device, sentence, text_field, tag_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]
        
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]

    unk_idx = text_field.vocab.stoi[text_field.unk_token]
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return tokens, predicted_tags, unks

In [41]:
example_index = 1

sentence = 'The Queen will deliver a speech about the conflict in North Korea at 1pm tomorrow.'


print(sentence)

The Queen will deliver a speech about the conflict in North Korea at 1pm tomorrow.


In [47]:
import spacy
tokens, pred_tags, unks = tag_sentence(model, 
                                       device, 
                                       sentence, 
                                       input_field, 
                                       label_field)

print(pred_tags)
print(tokens)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'O', 'O', 'O', 'O']
['the', 'queen', 'will', 'deliver', 'a', 'speech', 'about', 'the', 'conflict', 'in', 'north', 'korea', 'at', '1', 'pm', 'tomorrow', '.']


In [0]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

Pred. Tag	Actual Tag	Correct?	Token

PUNCT		PUNCT		✔		[
DET		DET		✔		this
NOUN		NOUN		✔		killing
ADP		ADP		✔		of
DET		DET		✔		a
ADJ		ADJ		✔		respected
NOUN		NOUN		✔		cleric
AUX		AUX		✔		will
AUX		AUX		✔		be
VERB		VERB		✔		causing
PRON		PRON		✔		us
NOUN		NOUN		✔		trouble
ADP		ADP		✔		for
NOUN		NOUN		✔		years
PART		PART		✔		to
VERB		VERB		✔		come
PUNCT		PUNCT		✔		.
PUNCT		PUNCT		✔		]


In [0]:
sentence = 'The Queen will deliver a speech about the conflict in North Korea at 1pm tomorrow.'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence, 
                                  TEXT, 
                                  UD_TAGS)

print(unks)

[]


In [0]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

Pred. Tag	Token

DET		the
NOUN		queen
AUX		will
VERB		deliver
DET		a
NOUN		speech
ADP		about
DET		the
NOUN		conflict
ADP		in
PROPN		north
PROPN		korea
ADP		at
NUM		1
NOUN		pm
NOUN		tomorrow
PUNCT		.


In [0]:
device = torch.device("cpu")
def loss_fn(outputs, labels):
  TAG_PAD_IDX =text_vocab.stoi[input_field.pad_token]
  criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
  criterion = criterion.to(device)
  scores = criterion(outputs, labels)
  return scores

def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.to(device)
    
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.inputs_word
       
        target = batch.labels
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            text = text.to(device)
            target = target.to(device)
        if (text.size()[0] is not 64):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        prediction = model(text)
        loss = loss_fn(prediction, target)
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.inputs_word
            if (text.size()[0] is not 64):
                continue
            target = batch.labels
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            target = target.view(-1)
            
            loss = loss_fn(predictions, target)
            num_corrects = (torch.max(predictions, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)
	



In [87]:
learning_rate = 2e-5
batch_size = 64


for epoch in range(10):
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

Epoch: 01, Train Loss: 0.000, Train Acc: 0.00%, Val. Loss: 0.000000, Val. Acc: 0.00%
Epoch: 02, Train Loss: 0.000, Train Acc: 0.00%, Val. Loss: 0.000000, Val. Acc: 0.00%
Epoch: 03, Train Loss: 0.000, Train Acc: 0.00%, Val. Loss: 0.000000, Val. Acc: 0.00%


KeyboardInterrupt: ignored

In [0]:
pred_cat = model.predict(X_te)
pred = np.argmax(pred_cat, axis=-1)
y_te_true = np.argmax(y_te, -1)
from sklearn_crfsuite.metrics import flat_classification_report

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in pred]
y_te_true_tag = [[idx2tag[i] for i in row] for row in y_te_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_te_true_tag)
print(report)

3

In [0]:
def zero_digits(s):
    """
    Replace every digit in a string by a zero.
    """
    return re.sub('\d', '0', s)

def load_sentences(path, zeros):
    """
    Load sentences. A line must contain at least a word and its tag.
    Sentences are separated by empty lines.
    """

    
    sentences = []
    sentence = []
    for line in codecs.open(path, 'r', 'utf8'):
        line = zero_digits(line.rstrip()) if zeros else line.rstrip()
        if not line:
            if len(sentence) > 0:
                if 'DOCSTART' not in sentence[0][0]:
                    sentences.append(sentence)
                sentence = []
        else:
            word = line.split()
            assert len(word) >= 2
            sentence.append(word)
    if len(sentence) > 0:
        if 'DOCSTART' not in sentence[0][0]:
            sentences.append(sentence)
    return sentences

In [0]:
train_sentences = load_sentences('/content/NER/corpus/CoNLL-2003/eng.train',zeros=True)
test_sentences = load_sentences('/content/NER/corpus/CoNLL-2003/eng.testb', zeros=True)
dev_sentences = load_sentences('/content/NER/corpus/CoNLL-2003/eng.testa', zeros=True)

In [0]:
len(train_sentences)

14041

In [0]:
train_sentences[2]

[['BRUSSELS', 'NNP', 'I-NP', 'I-LOC'], ['0000-00-00', 'CD', 'I-NP', 'O']]

In [0]:
def lower_case(x,lower=False):
    if lower:
        return x.lower()  
    else:
        return x

def prepare_dataset(sentences, word_to_id, char_to_id, tag_to_id, lower=False):
    """
    Prepare the dataset. Return a list of lists of dictionaries containing:
        - word indexes
        - word char indexes
        - tag indexes
    """
    data = []
    for s in sentences:
        str_words = [w[0] for w in s]
        words = [word_to_id[lower_case(w,lower) if lower_case(w,lower) in word_to_id else '<UNK>']
                 for w in str_words]
        # Skip characters that are not in the training set
        chars = [[char_to_id[c] for c in w if c in char_to_id]
                 for w in str_words]
        tags = [tag_to_id[w[-1]] for w in s]
        data.append({
            'str_words': str_words,
            'words': words,
            'chars': chars,
            'tags': tags,
        })
    return data

train_data = prepare_dataset(
    train_sentences, word_to_id, char_to_id, tag_to_id, parameters['lower']
)
dev_data = prepare_dataset(
    dev_sentences, word_to_id, char_to_id, tag_to_id, parameters['lower']
)
test_data = prepare_dataset(
    test_sentences, word_to_id, char_to_id, tag_to_id, parameters['lower']
)
print("{} / {} / {} sentences in train / dev / test.".format(len(train_data), len(dev_data), len(test_data)))

NameError: ignored